In [0]:
!git clone https://github.com/huggingface/transformers.git
%cd transformers/
!git checkout 896a0eb1fd861bc37097a9b669ebf4cb8d523de7
%cd ..
!git clone https://github.com/joyousprakhar/ICHI-dataset.git
!cp -a ICHI-dataset/. transformers/
%cd transformers/
!mkdir examples/ichi/
!pip install -r requirements.txt
!pip install .
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

Cloning into 'transformers'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 23780 (delta 3), reused 12 (delta 0), pack-reused 23750
Receiving objects: 100% (23780/23780), 14.25 MiB | 23.54 MiB/s, done.
Resolving deltas: 100% (16800/16800), done.
/content/transformers
Note: checking out '896a0eb1fd861bc37097a9b669ebf4cb8d523de7'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 896a0eb1 Merge pull request #2459 from Perseus14/patch-4
/content
Cloning into 'ICHI-dataset'...
remote: Enumerating objects: 41, done.
remote: Cou

True

In [0]:
########### Uncomment it to download Glove Vectors

!curl -O -J -L http://nlp.stanford.edu/data/glove.840B.300d.zip
!unzip glove.840B.300d.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0   315    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0   352    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 2075M  100 2075M    0     0  2098k      0  0:16:52  0:16:52 --:--:-- 2659k
Archive:  glove.840B.300d.zip
  inflating: glove.840B.300d.txt     


In [0]:
########### Uncomment it to download BioASQ Embeddings

# !curl -O -J -L http://bioasq.lip6.fr/tools/BioASQword2vec/
# !tar -xvzf biomedicalWordVectors.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   345  100   345    0     0    442      0 --:--:-- --:--:-- --:--:--   441
100 1279M  100 1279M    0     0  15.8M      0  0:01:20  0:01:20 --:--:-- 17.0M
curl: Saved to filename 'biomedicalWordVectors.tar.gz'
word2vecTools/toolkit.py
word2vecTools/vectors.txt
word2vecTools/
word2vecTools/README_BioASQ_word_vectors.pdf
word2vecTools/types.txt
word2vecTools/train_vectors.sh


In [0]:
# class ICHIDataset(Dataset):
#     def __init__(self, fname, tokenizer):
#         fin = open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
#         lines = fin.readlines()
#         fin.close()

#         all_data = []
#         for i in range(0, len(lines), 3):
#             text_left, _, text_right = [s.lower().strip() for s in lines[i].partition("$T$")]
#             aspect = lines[i + 1].lower().strip()
#             polarity = lines[i + 2].strip()

#             text_raw_indices = tokenizer.text_to_sequence(text_left + " " + aspect + " " + text_right)
#             text_raw_without_aspect_indices = tokenizer.text_to_sequence(text_left + " " + text_right)
#             text_left_indices = tokenizer.text_to_sequence(text_left)
#             text_left_with_aspect_indices = tokenizer.text_to_sequence(text_left + " " + aspect)
#             text_right_indices = tokenizer.text_to_sequence(text_right, reverse=True)
#             text_right_with_aspect_indices = tokenizer.text_to_sequence(" " + aspect + " " + text_right, reverse=True)
#             aspect_indices = tokenizer.text_to_sequence(aspect)
#             left_context_len = np.sum(text_left_indices != 0)
#             aspect_len = np.sum(aspect_indices != 0)
#             aspect_in_text = torch.tensor([left_context_len.item(), (left_context_len + aspect_len - 1).item()])
#             polarity = int(polarity) + 1

#             text_bert_indices = tokenizer.text_to_sequence('[CLS] ' + text_left + " " + aspect + " " + text_right + ' [SEP] ' + aspect + " [SEP]")
#             bert_segments_ids = np.asarray([0] * (np.sum(text_raw_indices != 0) + 2) + [1] * (aspect_len + 1))
#             bert_segments_ids = pad_and_truncate(bert_segments_ids, tokenizer.max_seq_len)

#             text_raw_bert_indices = tokenizer.text_to_sequence("[CLS] " + text_left + " " + aspect + " " + text_right + " [SEP]")
#             aspect_bert_indices = tokenizer.text_to_sequence("[CLS] " + aspect + " [SEP]")

#             data = {
#                 'text_bert_indices': text_bert_indices,
#                 'bert_segments_ids': bert_segments_ids,
#                 'text_raw_bert_indices': text_raw_bert_indices,
#                 'aspect_bert_indices': aspect_bert_indices,
#                 'text_raw_indices': text_raw_indices,
#                 'text_raw_without_aspect_indices': text_raw_without_aspect_indices,
#                 'text_left_indices': text_left_indices,
#                 'text_left_with_aspect_indices': text_left_with_aspect_indices,
#                 'text_right_indices': text_right_indices,
#                 'text_right_with_aspect_indices': text_right_with_aspect_indices,
#                 'aspect_indices': aspect_indices,
#                 'aspect_in_text': aspect_in_text,
#                 'polarity': polarity,
#             }

#             all_data.append(data)
#         self.data = all_data

#     def __getitem__(self, index):
#         return self.data[index]

#     def __len__(self):
#         return len(self.data)

In [0]:
!pwd

/content/transformers


In [0]:
%%writefile /content/transformers/examples/ichi/utils_ichi.py
import logging
import os
import torch
import random
import re
import csv
import pickle
import numpy as np
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from sklearn.metrics import matthews_corrcoef, f1_score
from transformers.file_utils import is_tf_available 
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

if is_tf_available():
    import tensorflow as tf

logger = logging.getLogger(__name__)

def simple_accuracy(preds, labels):
    return (preds == labels).mean()

def acc_and_f1(preds, labels):
    acc = simple_accuracy(preds, labels)
    f1 = f1_score(y_true=labels, y_pred=preds)
    return {
        "acc": acc,
        "f1": f1,
        "acc_and_f1": (acc + f1) / 2,
    }

def compute_metrics(preds, labels):
    return {"acc": simple_accuracy(preds, labels)}


def pad_and_truncate(sequence, maxlen, dtype='int64', padding='post', truncating='post', value=0):
    x = (np.ones(maxlen) * value).astype(dtype)
    if truncating == 'pre':
        trunc = sequence[-maxlen:]
    else:
        trunc = sequence[:maxlen]
    trunc = np.asarray(trunc, dtype=dtype)
    if padding == 'post':
        x[:len(trunc)] = trunc
    else:
        x[-len(trunc):] = trunc
    return x

class Tokenizer(object):
    def __init__(self, max_seq_len, lower=True):
        self.lower = lower
        self.max_seq_len = max_seq_len
        self.word2idx = {}
        self.idx2word = {}
        self.idx2word[0] = '<PAD>'
        self.word2idx['<PAD>'] = 0
        self.idx = 1

    def fit_on_text(self, text):
        if self.lower:
            text = text.lower()
        words = text.split()
        for word in words:
            if word not in self.word2idx:
                self.word2idx[word] = self.idx
                self.idx2word[self.idx] = word
                self.idx += 1

    def fit_on_examples(self, examples):
        is_tf_dataset = False
        if is_tf_available() and isinstance(examples, tf.data.Dataset):
            is_tf_dataset = True
        processor = ICHIProcessor()
        for example in examples:
            if is_tf_dataset:
                example = processor.get_example_from_tensor_dict(example)
                example = processor.tfds_map(example)
            self.fit_on_text(example.clean_text)

    def text_to_sequence(self, text, reverse=False, padding='post', truncating='post'):
        if self.lower:
            text = text.lower()
        words = text.split()
        unknownidx = len(self.word2idx)+1
        sequence = [self.word2idx[w] if w in self.word2idx else unknownidx for w in words]
        if len(sequence) == 0:
            sequence = [0]
        if reverse:
            sequence = sequence[::-1]
        return pad_and_truncate(sequence, self.max_seq_len, padding=padding, truncating=truncating)

def _load_word_vec_glove(path, word2idx=None):
    fin = open(path, 'r', encoding='utf-8', newline='\n', errors='ignore')
    word_vec = {}
    for line in fin:
        tokens = line.rstrip().split()
        if word2idx is None or tokens[0] in word2idx.keys():
            try:
                word_vec[tokens[0]] = np.asarray(tokens[1:], dtype='float32')
            except:
                pass
    return word_vec
    
def build_embedding_matrix_glove(word2idx, embed_dim, dat_fname, fname):
    if os.path.exists(dat_fname):
        print('loading embedding_matrix(glove): ', dat_fname)
        embedding_matrix = pickle.load(open(dat_fname, 'rb'))
    else:
        print('loading word vectors...(glove)')
        embedding_matrix = np.zeros((len(word2idx) + 2, embed_dim))  # idx 0 and len(word2idx)+1 are all-zeros
        fname = fname + '/glove.twitter.27B/glove.twitter.27B.' + str(embed_dim) + 'd.txt' \
            if embed_dim != 300 else fname + '/glove.840B.300d.txt'
        word_vec = _load_word_vec_glove(fname, word2idx=word2idx)
        print('building embedding_matrix:', dat_fname)
        for word, i in word2idx.items():
            vec = word_vec.get(word)
            if vec is not None:
                # words not found in embedding index will be all-zeros.
                embedding_matrix[i] = vec
        pickle.dump(embedding_matrix, open(dat_fname, 'wb'))
    return embedding_matrix

def build_embedding_matrix_BioASQ(word2idx, embed_dim, dat_fname, fname):
    if os.path.exists(dat_fname):
        print('loading embedding_matrix(BioASQ):', dat_fname)
        embedding_matrix = pickle.load(open(dat_fname, 'rb'))
    else:
        print('loading word vectors...(BioASQ)')
        embedding_matrix = np.zeros((len(word2idx) + 2, embed_dim))  # idx 0 and len(word2idx)+1 are all-zeros
        f = open(fname + "/word2vecTools/types.txt","r")
        i = 0
        names = []
        for line in f:
            names.append(line.split('\n')[0])
            i = i + 1
        vectors = np.loadtxt(fname + "/word2vecTools/vectors.txt")
        word_vec = {}
        for (index, name) in enumerate(names):
            word_vec[name] = index

        print('building embedding_matrix:', dat_fname)
        for word, i in word2idx.items():
            if word in word_vec.keys():
                vec = vectors[word_vec[word]]
            else:
                vec = None
            if vec is not None:
                # words not found in embedding index will be all-zeros.
                embedding_matrix[i] = vec
        pickle.dump(embedding_matrix, open(dat_fname, 'wb'))
    return embedding_matrix

def convert_examples_to_features(
    examples,
    tokenizer,
    tokenizer_cleantext,
    max_length=512,
    task=None,
    label_list=None,
    output_mode=None,
    pad_on_left=False,
    pad_token=0,
    pad_token_segment_id=0,
    mask_padding_with_zero=True,
):
    """
    Loads a data file into a list of ``InputFeatures``

    Args:
        examples: List of ``InputExamples`` or ``tf.data.Dataset`` containing the examples.
        tokenizer: Instance of a tokenizer that will tokenize the examples
        tokenizer_cleantext: Instance of a tokenizer that will tokenize the examples clean text(used for our medical module) 
        max_length: Maximum example length
        task: GLUE task
        label_list: List of labels. Can be obtained from the processor using the ``processor.get_labels()`` method
        output_mode: String indicating the output mode. Either ``regression`` or ``classification``
        pad_on_left: If set to ``True``, the examples will be padded on the left rather than on the right (default)
        pad_token: Padding token
        pad_token_segment_id: The segment ID for the padding token (It is usually 0, but can vary such as for XLNet where it is 4)
        mask_padding_with_zero: If set to ``True``, the attention mask will be filled by ``1`` for actual values
            and by ``0`` for padded values. If set to ``False``, inverts it (``1`` for padded values, ``0`` for
            actual values)

    Returns:
        If the ``examples`` input is a ``tf.data.Dataset``, will return a ``tf.data.Dataset``
        containing the task-specific features. If the input is a list of ``InputExamples``, will return
        a list of task-specific ``InputFeatures`` which can be fed to the model.

    """
    is_tf_dataset = False
    if is_tf_available() and isinstance(examples, tf.data.Dataset):
        is_tf_dataset = True

    if task is not None:
        processor = ICHIProcessor()
        if label_list is None:
            label_list = processor.get_labels()
            logger.info("Using label list %s for task %s" % (label_list, task))
        if output_mode is None:
            output_mode = "classification"
            logger.info("Using output mode %s for task %s" % (output_mode, task))

    label_map = {label: i for i, label in enumerate(label_list)}

    features = []
    for (ex_index, example) in enumerate(examples):
        if ex_index % 10000 == 0:
            logger.info("Writing example %d/%d" % (ex_index, len(examples)))
        if is_tf_dataset:
            example = processor.get_example_from_tensor_dict(example)
            example = processor.tfds_map(example)

        aspect_present = bool(example.aspects is not None)
        if aspect_present:
            if 4 * len(example.aspects) > max_length:
                example.aspects = random.sample(example.aspects, k = int(max_length/4))
                

        inputs_global = tokenizer.encode_plus_lcf(example.text, example.heading, example.aspects, add_special_tokens=True, max_length=max_length,) ######## edittttttt
        inputs_local = tokenizer.encode_plus_lcf(example.text, add_special_tokens=True, max_length=max_length,)
        text_clean_indices = tokenizer_cleantext.text_to_sequence(example.clean_text)
        if len(example.aspects) > 2:
            aspect_indices = tokenizer.encode_plus_lcf(example.aspects[0], example.aspects[1], example.aspects[2:], add_special_tokens=False, max_length=max_length,)
        elif len(example.aspects) == 2:
            aspect_indices = tokenizer.encode_plus_lcf(example.aspects[0], example.aspects[1], add_special_tokens=False, max_length=max_length,)
        else:
            aspect_indices = tokenizer.encode_plus_lcf(example.aspects[0], add_special_tokens=False, max_length=max_length,)
        
        input_global_ids, token_global_type_ids = inputs_global["input_ids"], inputs_global["token_type_ids"]
        input_local_ids, token_local_type_ids = inputs_local["input_ids"], inputs_local["token_type_ids"]
        
        aspect_indices = aspect_indices["input_ids"]
        padding_length_aspect = max_length - len(aspect_indices)
        aspect_indices = aspect_indices + ([pad_token] * padding_length_aspect)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        attention_mask_global = [1 if mask_padding_with_zero else 0] * len(input_global_ids)
        attention_mask_local = [1 if mask_padding_with_zero else 0] * len(input_local_ids)

        # Zero-pad up to the sequence length.
        padding_length_global = max_length - len(input_global_ids)
        padding_length_local = max_length - len(input_local_ids)
        if pad_on_left:
            input_global_ids = ([pad_token] * padding_length_global) + input_global_ids
            attention_mask_global = ([0 if mask_padding_with_zero else 1] * padding_length_global) + attention_mask_global
            token_global_type_ids = ([pad_token_segment_id] * padding_length_global) + token_global_type_ids
            input_local_ids = ([pad_token] * padding_length_local) + input_local_ids
            attention_mask_local = ([0 if mask_padding_with_zero else 1] * padding_length_local) + attention_mask_local
            token_local_type_ids = ([pad_token_segment_id] * padding_length_local) + token_local_type_ids
        else:
            input_global_ids = input_global_ids + ([pad_token] * padding_length_global)
            attention_mask_global = attention_mask_global + ([0 if mask_padding_with_zero else 1] * padding_length_global)
            token_global_type_ids = token_global_type_ids + ([pad_token_segment_id] * padding_length_global)
            input_local_ids = input_local_ids + ([pad_token] * padding_length_local)
            attention_mask_local = attention_mask_local + ([0 if mask_padding_with_zero else 1] * padding_length_local)
            token_local_type_ids = token_local_type_ids + ([pad_token_segment_id] * padding_length_local)

        assert len(input_global_ids) == max_length, "Error with input length {} vs {}".format(len(input_global_ids), max_length)
        assert len(input_local_ids) == max_length, "Error with input length {} vs {}".format(len(input_local_ids), max_length)
        assert len(attention_mask_global) == max_length, "Error with input length {} vs {}".format(
            len(attention_mask_global), max_length
        )
        assert len(attention_mask_local) == max_length, "Error with input length {} vs {}".format(
            len(attention_mask_local), max_length
        )
        assert len(token_global_type_ids) == max_length, "Error with input length {} vs {}".format(
            len(token_global_type_ids), max_length
        )
        assert len(token_local_type_ids) == max_length, "Error with input length {} vs {}".format(
            len(token_local_type_ids), max_length
        )

        if output_mode == "classification":
            label = label_map[example.label]
        elif output_mode == "regression":
            label = float(example.label)
        else:
            raise KeyError(output_mode)

        if ex_index < 5:
            logger.info("*** Example ***")
            logger.info("guid: %s" % (example.guid))
            logger.info("input_global_ids: %s" % " ".join([str(x) for x in input_global_ids]))
            logger.info("attention_mask_global: %s" % " ".join([str(x) for x in attention_mask_global]))
            logger.info("token_global_type_ids: %s" % " ".join([str(x) for x in token_global_type_ids]))
            logger.info("input_local_ids: %s" % " ".join([str(x) for x in input_local_ids]))
            logger.info("attention_mask_local: %s" % " ".join([str(x) for x in attention_mask_local]))
            logger.info("token_local_type_ids: %s" % " ".join([str(x) for x in token_local_type_ids]))
            logger.info("text_clean_indices: %s" % " ".join([str(x) for x in text_clean_indices]))
            logger.info("aspect_indices: %s" % " ".join([str(x) for x in aspect_indices]))
            logger.info("label: %s (id = %d)" % (example.label, label))

        features.append(
            ICHI_InputFeatures(
                input_global_ids=input_global_ids, input_local_ids=input_local_ids, attention_mask_global=attention_mask_global, attention_mask_local=attention_mask_local,
                token_global_type_ids=token_global_type_ids, token_local_type_ids=token_local_type_ids, text_clean_indices=text_clean_indices, aspect_indices=aspect_indices, label=label
            )
        )

    if is_tf_available() and is_tf_dataset:

        def gen():
            for ex in features:
                yield (
                    {
                        "input_global_ids": ex.input_global_ids,
                        "input_local_ids": ex.input_local_ids,
                        "attention_mask_global": ex.attention_mask_global,
                        "attention_mask_local": ex.attention_mask_local,
                        "token_global_type_ids": ex.token_global_type_ids,
                        "token_local_type_ids": ex.token_local_type_ids,
                        "text_clean_indices": ex.text_clean_indices,
                        "aspect_indices": ex.aspect_indices,
                    },
                    ex.label,
                )

        return tf.data.Dataset.from_generator(
            gen,
            ({"input_global_ids": tf.int32, "input_local_ids": tf.int32, "attention_mask_global": tf.int32, "attention_mask_local": tf.int32, "token_global_type_ids": tf.int32, "token_local_type_ids": tf.int32, "text_clean_indices": tf.int32, "aspect_indices": tf.int32}, tf.int64),
            (
                {
                    "input_global_ids": tf.TensorShape([None]),
                    "input_local_ids": tf.TensorShape([None]),
                    "attention_mask_global": tf.TensorShape([None]),
                    "attention_mask_local": tf.TensorShape([None]),
                    "token_global_type_ids": tf.TensorShape([None]),
                    "token_local_type_ids": tf.TensorShape([None]),
                    "text_clean_indices": tf.TensorShape([None]),
                    "aspect_indices": tf.TensorShape([None]),
                },
                tf.TensorShape([]),
            ),
        )

    return features

class ICHIProcessor(object):
    """Processor for the ICHI data set (GLUE version)."""

    def get_example_from_tensor_dict(self, tensor_dict):
        """See base class."""
        return ICHI_InputExample(tensor_dict['idx'].numpy(),
                            tensor_dict['heading'].numpy().decode('utf-8'),
                            tensor_dict['text'].numpy().decode('utf-8'),
                            tensor_dict['clean_text'].numpy().decode('utf-8'),
                            tensor_dict['aspects'].numpy().decode('utf-8'),
                            str(tensor_dict['label'].numpy()))

    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "dev.tsv")),
            "dev")

    def get_labels(self):
        """See base class."""
        return ["DEMO", "DISE", "TRMT", "GOAL", "PREG", "FAML", "SOCL"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training and dev sets."""
        examples = []
        lemmatizer = WordNetLemmatizer()
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            guid = "%s-%s" % (set_type, i)
            label = line[0]
            heading = None
            text = line[1] + ' ' + line[2]
            try:
                aspects = [' ' + x for x in line[3].split('|')]
                temp_clean_text = " ".join(line[3].split('|'))
            except:
                aspects = None
                temp_clean_text = ""
                print("Sed")
            
            # print(aspects)
            clean_text = self.clean_str( text, lemmatizer) ######## for using glove embeddings over sentence
            # clean_text = self.clean_str( temp_clean_text, lemmatizer) ##### for using BioASQ embeddings in aspects
            examples.append(
                ICHI_InputExample(guid=guid, heading=heading, text=text, clean_text=clean_text, aspects=aspects, label=label))
        return examples

    def tfds_map(self, example):
        """Some tensorflow_datasets datasets are not formatted the same way the GLUE datasets are.
        This method converts examples to the correct format."""
        if len(self.get_labels()) > 1:
            example.label = self.get_labels()[int(example.label)]
        return example

    def clean_str(self, string1, lemmatizer):
        """
        Tokenization/string cleaning for dataset
        Every dataset is lower cased except
        """
        str_stop = ""
        string1 = re.sub(r'^https?:\/\/.*[\r\n]*', '',string1)
        string1 = re.sub(r"\\", " ", string1)    
        string1 = re.sub(r"\'", " ", string1)    
        string1 = re.sub(r"\"", " ", string1)   
        string1 = re.sub(r'(\W)\1+', r'\1', string1)
        word_list=string1.split(" ")
        filtered_words = [word for word in word_list if word not in stopwords.words('english')]
        for kj in filtered_words:
            new=lemmatizer.lemmatize(str(kj)) 
            str_stop=str_stop +" "+new
            str_stop.encode('utf-8')
        return str_stop.strip().lower()

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding="utf-8-sig") as f:
            return list(csv.reader(f, delimiter="\t", quotechar=quotechar))

class ICHI_InputExample(object):
    """
    A single training/test example for simple sequence classification.

    Args:
        guid: Unique id for the example.
        text_a: string. The untokenized text of the first sequence. For single
        sequence tasks, only this sequence must be specified.
        text_b: (Optional) string. The untokenized text of the second sequence.
        Only must be specified for sequence pair tasks.
        label: (Optional) string. The label of the example. This should be
        specified for train and dev examples, but not for test examples.
    """

    def __init__(self, guid, heading, text, clean_text, aspects, label=None):
        self.guid = guid
        self.heading = heading
        self.text = text
        self.clean_text = clean_text
        self.aspects = aspects
        self.label = label

    def __repr__(self):
        return str(self.to_json_string())

    def to_dict(self):
        """Serializes this instance to a Python dictionary."""
        output = copy.deepcopy(self.__dict__)
        return output

    def to_json_string(self):
        """Serializes this instance to a JSON string."""
        return json.dumps(self.to_dict(), indent=2, sort_keys=True) + "\n"


class ICHI_InputFeatures(object):
    """
    A single set of features of data.

    Args:
        input_ids: Indices of input sequence tokens in the vocabulary.
        attention_mask: Mask to avoid performing attention on padding token indices.
            Mask values selected in ``[0, 1]``:
            Usually  ``1`` for tokens that are NOT MASKED, ``0`` for MASKED (padded) tokens.
        token_type_ids: Segment token indices to indicate first and second portions of the inputs.
        label: Label corresponding to the input
    """

    def __init__(self, input_global_ids, input_local_ids, attention_mask_global=None, attention_mask_local=None, token_global_type_ids=None, token_local_type_ids=None, text_clean_indices=None, aspect_indices=None, label=None):
        self.input_global_ids = input_global_ids
        self.attention_mask_global = attention_mask_global
        self.token_global_type_ids = token_global_type_ids
        self.input_local_ids = input_local_ids
        self.attention_mask_local = attention_mask_local
        self.token_local_type_ids = token_local_type_ids
        self.text_clean_indices = text_clean_indices
        self.aspect_indices = aspect_indices
        self.label = label

    def __repr__(self):
        return str(self.to_json_string())

    def to_dict(self):
        """Serializes this instance to a Python dictionary."""
        output = copy.deepcopy(self.__dict__)
        return output

    def to_json_string(self):
        """Serializes this instance to a JSON string."""
        return json.dumps(self.to_dict(), indent=2, sort_keys=True) + "\n"


def load_and_cache_examples(args, task, tokenizer, tokenizer_cleantext, evaluate=False):
    if args.local_rank not in [-1, 0] and not evaluate:
        torch.distributed.barrier()  # Make sure only the first process in distributed training process the dataset, and the others will use the cache

    processor = ICHIProcessor()  #################### update !!!!!!!!!!!!!!!!1
    output_mode = "classification"
    # Load data features from cache or dataset file
    cached_features_file = os.path.join(
        args.data_dir,
        "cached_{}_{}_{}_{}".format(
            "dev" if evaluate else "train",
            list(filter(None, args.model_name_or_path.split("/"))).pop(),
            str(args.max_seq_length),
            str(task),
        ),
    )
    cached_tokenizer_cleantext_file = os.path.join(
        args.data_dir,
        "cachedtokenizer_{}_{}_{}".format(
            list(filter(None, args.model_name_or_path.split("/"))).pop(),
            str(args.embedding_type),
            str(task),
        ),
    )
    
    if os.path.exists(cached_features_file) and os.path.exists(cached_tokenizer_cleantext_file) and not args.overwrite_cache:
        logger.info("Loading features from cached file %s", cached_features_file)
        features = torch.load(cached_features_file)
        print('loading tokenizer from cache:', cached_tokenizer_cleantext_file)
        tokenizer_cleantext = pickle.load(open(cached_tokenizer_cleantext_file, 'rb'))
    else:
        logger.info("Creating features from dataset file at %s", args.data_dir)
        label_list = processor.get_labels()
        if task in ["mnli", "mnli-mm"] and args.model_type in ["roberta", "xlmroberta"]:
            # HACK(label indices are swapped in RoBERTa pretrained model)
            label_list[1], label_list[2] = label_list[2], label_list[1]
        examples = (
            processor.get_dev_examples(args.data_dir) if evaluate else processor.get_train_examples(args.data_dir)
        )
        if (not evaluate):
            tokenizer_cleantext.fit_on_examples(examples)
        ########################## update kar!!!!!!!!!!!!!!!!!!!!!!!!!!
        features = convert_examples_to_features(
            examples,
            tokenizer,
            tokenizer_cleantext,
            label_list=label_list,
            max_length=args.max_seq_length,
            output_mode=output_mode,
            pad_on_left=bool(args.model_type in ["xlnet"]),  # pad on the left for xlnet
            pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
            pad_token_segment_id=4 if args.model_type in ["xlnet"] else 0,
        )
        if args.local_rank in [-1, 0]:
            logger.info("Saving features into cached file %s", cached_features_file)
            torch.save(features, cached_features_file)
            pickle.dump(tokenizer_cleantext, open(cached_tokenizer_cleantext_file, 'wb'))

    if args.local_rank == 0 and not evaluate:
        torch.distributed.barrier()  # Make sure only the first process in distributed training process the dataset, and the others will use the cache

    # Convert to Tensors and build dataset
    all_input_global_ids = torch.tensor([f.input_global_ids for f in features], dtype=torch.long)
    all_input_local_ids = torch.tensor([f.input_local_ids for f in features], dtype=torch.long)
    all_attention_mask_global = torch.tensor([f.attention_mask_global for f in features], dtype=torch.long)
    all_attention_mask_local = torch.tensor([f.attention_mask_local for f in features], dtype=torch.long)
    all_token_global_type_ids = torch.tensor([f.token_global_type_ids for f in features], dtype=torch.long)
    all_token_local_type_ids = torch.tensor([f.token_local_type_ids for f in features], dtype=torch.long)
    all_text_clean_indices = torch.tensor([f.text_clean_indices for f in features], dtype=torch.long)
    all_aspect_indices = torch.tensor([f.aspect_indices for f in features], dtype=torch.long)
    all_labels = torch.tensor([f.label for f in features], dtype=torch.long)
    
    dataset = TensorDataset(all_input_global_ids, all_input_local_ids, all_attention_mask_global, all_attention_mask_local, all_token_global_type_ids, all_token_local_type_ids, all_text_clean_indices, all_aspect_indices, all_labels)
    
    return dataset, tokenizer_cleantext

Overwriting /content/transformers/examples/ichi/utils_ichi.py


In [0]:
%%writefile /content/transformers/examples/ichi/lcf_ichi.py
import torch
import torch.nn as nn
import copy
import numpy as np
from transformers.modeling_bert import BertPooler, BertSelfAttention, BertPreTrainedModel, BertModel
from torch.nn import CrossEntropyLoss, MSELoss

class SelfAttention(nn.Module):
    def __init__(self, config, args):
        super(SelfAttention, self).__init__()
        self.args = args
        self.SA = BertSelfAttention(config)
        self.tanh = torch.nn.Tanh()

    def forward(self, inputs):
        zero_tensor = torch.tensor(np.zeros((inputs.size(0), 1, 1, self.args.max_seq_length),
                                            dtype=np.float32), dtype=torch.float32).to(self.args.device)
        SA_out = self.SA(inputs, zero_tensor)
        return self.tanh(SA_out[0])


class lcf_BERT(BertPreTrainedModel):
    def __init__(self, config, args):
        super(lcf_BERT, self).__init__(config)
        self.num_labels = config.num_labels
        self.args = args
        self.config =config
        self.bert = BertModel(config)
        # self.bert_global_focus = self.bert
        # self.bert_local_focus = copy.deepcopy(self.bert_global_focus) if args.use_single_bert else self.bert_global_focus
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.bert_SA = SelfAttention(config, args) ### change
        self.linear_double_cdm_or_cdw = nn.Linear(config.hidden_size * 2,config.hidden_size)
        self.linear_triple_lcf_global = nn.Linear(config.hidden_size * 3, config.hidden_size)
        self.bert_pooler = BertPooler(config) ### change
        self.dense = nn.Linear(config.hidden_size, config.num_labels)
        self.init_weights()

    def feature_dynamic_mask(self, text_local_indices, aspect_indices):
        texts = text_local_indices
        # mask_len = self.args.SRD
        masked_text_raw_indices = torch.zeros((text_local_indices.size(0), self.args.max_seq_length, self.config.hidden_size),
                                          dtype=torch.float)
        
        zero_tensor = torch.tensor(0).to(self.args.device)
        for i in range(aspect_indices.shape[0]):
            for j in range(aspect_indices[i].shape[0]):
                if aspect_indices[i][j] == zero_tensor:
                    break
                else:
                    indices = (text_local_indices[i] == aspect_indices[i][j]).nonzero()
                    for k in indices:
                        masked_text_raw_indices[i][k] = torch.ones(self.config.hidden_size, dtype=torch.float)
        return masked_text_raw_indices.to(self.args.device)

    # create the weights tensor for local context features
    def feature_dynamic_weighted(self, text_local_indices, aspect_indices):
        texts = text_local_indices
        asps = aspect_indices
        # mask_len = self.args.SRD
        masked_text_raw_indices = torch.zeros((text_local_indices.size(0), self.args.max_seq_length, self.config.hidden_size),
                                          dtype=torch.float)
        for text_i, asp_i in zip(range(len(texts)), range(len(asps))):
            asp_len = np.count_nonzero(asps[asp_i]) - 2
            try:
                asp_begin = np.argwhere(texts[text_i] == asps[asp_i][1])[0][0]
                asp_avg_index = (asp_begin * 2 + asp_len) / 2
            except:
                continue
            distances = np.zeros(np.count_nonzero(texts[text_i]), dtype=np.float32)
            for i in range(1, np.count_nonzero(texts[text_i])-1):
                if abs(i - asp_avg_index) + asp_len / 2 > self.args.SRD:
                    distances[i] = 1 - (abs(i - asp_avg_index)+asp_len/2
                                        - self.args.SRD)/np.count_nonzero(texts[text_i])
                else:
                    distances[i] = 1
            for i in range(len(distances)):
                masked_text_raw_indices[text_i][i] = masked_text_raw_indices[text_i][i] * distances[i]
        # masked_text_raw_indices = torch.from_numpy(masked_text_raw_indices)
        return masked_text_raw_indices.to(self.args.device)

    def forward(
        self,
        input_global_ids=None,
        attention_mask_global=None,
        token_global_type_ids=None,
        input_local_ids=None,
        attention_mask_local=None,
        token_local_type_ids=None,
        text_clean_indices=None,
        aspect_indices=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
    ):

        global_outputs, _ = self.bert(
            input_global_ids,
            attention_mask=attention_mask_global,
            token_type_ids=token_global_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )

        local_outputs, _ = self.bert(
            input_local_ids,
            attention_mask=attention_mask_local,
            token_type_ids=token_local_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )

        # out_cat = torch.cat((local_outputs, global_outputs), dim=-1)
        # out_cat = self.linear_double_cdm_or_cdw(out_cat)
        
        if self.args.local_context_focus == 'cdm':
            masked_local_text_vec = self.feature_dynamic_mask(input_local_ids, aspect_indices)
            local_outputs = torch.mul(local_outputs, masked_local_text_vec)
            out_cat = torch.cat((local_outputs, global_outputs), dim=-1)
            out_cat = self.linear_double_cdm_or_cdw(out_cat)

        elif self.args.local_context_focus == 'cdw':
            weighted_text_local_features = self.feature_dynamic_weighted(input_local_ids, aspect_indices)
            local_outputs = torch.mul(local_outputs, weighted_text_local_features)
            out_cat = torch.cat((local_outputs, global_outputs), dim=-1)
            out_cat = self.linear_double_cdm_or_cdw(out_cat)

        elif self.args.local_context_focus == 'lcf_fusion':
            masked_local_text_vec = self.feature_dynamic_mask(text_local_indices, aspect_indices)
            masked_local_out = torch.mul(local_outputs, masked_local_text_vec)
            weighted_text_local_features = self.feature_dynamic_weighted(text_local_indices, aspect_indices)
            weighted_local_out = torch.mul(local_outputs, weighted_text_local_features)
            out_cat = torch.cat((masked_local_out, global_outputs, weighted_local_out), dim=-1)
            out_cat = self.linear_triple_lcf_global(out_cat)

        self_attention_out = self.bert_SA(out_cat)
        self_attention_out = self.dropout(self_attention_out)
        pooled_out = self.bert_pooler(self_attention_out)
        logits = self.dense(pooled_out)
        outputs = (logits,)

        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs

Overwriting /content/transformers/examples/ichi/lcf_ichi.py


In [0]:
%%writefile /content/transformers/examples/ichi/run_ichi.py
import argparse
import glob
import json
import logging
import os
import random
import pickle

import numpy as np
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm, trange

from transformers import (
    WEIGHTS_NAME,
    AdamW,
    AlbertConfig,
    AlbertForSequenceClassification,
    AlbertTokenizer,
    BertConfig,
    BertForSequenceClassification,
    BertTokenizer,
    DistilBertConfig,
    DistilBertForSequenceClassification,
    DistilBertTokenizer,
    RobertaConfig,
    RobertaForSequenceClassification,
    RobertaTokenizer,
    XLMConfig,
    XLMForSequenceClassification,
    XLMRobertaConfig,
    XLMRobertaForSequenceClassification,
    XLMRobertaTokenizer,
    XLMTokenizer,
    XLNetConfig,
    XLNetForSequenceClassification,
    XLNetTokenizer,
    get_linear_schedule_with_warmup,
)
from lcf_ichi import lcf_BERT#, lcf_XLNET, lcf_XLM, lcf_Roberta, lcf_DistilBert, lcf_Albert, lcf_XLMRoberta
from utils_ichi import convert_examples_to_features, ICHIProcessor, compute_metrics, load_and_cache_examples, Tokenizer, pad_and_truncate, build_embedding_matrix_glove, build_embedding_matrix_BioASQ

try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter


logger = logging.getLogger(__name__)

ALL_MODELS = sum(
    (
        tuple(conf.pretrained_config_archive_map.keys())
        for conf in (
            BertConfig,
            XLNetConfig,
            XLMConfig,
            RobertaConfig,
            DistilBertConfig,
            AlbertConfig,
            XLMRobertaConfig,
        )
    ),
    (),
)

MODEL_CLASSES = {
    "bert": (BertConfig, BertForSequenceClassification, BertTokenizer, lcf_BERT),
}
#     "xlnet": (XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer, lcf_XLNET),
#     "xlm": (XLMConfig, XLMForSequenceClassification, XLMTokenizer, lcf_XLM),
#     "roberta": (RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer, lcf_Roberta),
#     "distilbert": (DistilBertConfig, DistilBertForSequenceClassification, DistilBertTokenizer, lcf_DistilBert),
#     "albert": (AlbertConfig, AlbertForSequenceClassification, AlbertTokenizer, lcf_Albert),
#     "xlmroberta": (XLMRobertaConfig, XLMRobertaForSequenceClassification, XLMRobertaTokenizer, lcf_XLMRoberta),
# }


def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)


def train(args, train_dataset, model, tokenizer, tokenizer_cleantext):
    """ Train the model """
    if args.local_rank in [-1, 0]:
        tb_writer = SummaryWriter()

    args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)
    train_sampler = RandomSampler(train_dataset) if args.local_rank == -1 else DistributedSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.train_batch_size)

    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]

    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
    )

    # Check if saved optimizer or scheduler states exist
    if os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt")) and os.path.isfile(
        os.path.join(args.model_name_or_path, "scheduler.pt")
    ):
        # Load in optimizer and scheduler states
        optimizer.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "optimizer.pt")))
        scheduler.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "scheduler.pt")))

    if args.fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)

    # multi-gpu training (should be after apex fp16 initialization)
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Distributed training (should be after apex fp16 initialization)
    if args.local_rank != -1:
        model = torch.nn.parallel.DistributedDataParallel(
            model, device_ids=[args.local_rank], output_device=args.local_rank, find_unused_parameters=True,
        )

    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  Instantaneous batch size per GPU = %d", args.per_gpu_train_batch_size)
    logger.info(
        "  Total train batch size (w. parallel, distributed & accumulation) = %d",
        args.train_batch_size
        * args.gradient_accumulation_steps
        * (torch.distributed.get_world_size() if args.local_rank != -1 else 1),
    )
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    epochs_trained = 0
    steps_trained_in_current_epoch = 0
    # Check if continuing training from a checkpoint
    if os.path.exists(args.model_name_or_path):
        # set global_step to gobal_step of last saved checkpoint from model path
        global_step = int(args.model_name_or_path.split("-")[-1].split("/")[0])
        epochs_trained = global_step // (len(train_dataloader) // args.gradient_accumulation_steps)
        steps_trained_in_current_epoch = global_step % (len(train_dataloader) // args.gradient_accumulation_steps)

        logger.info("  Continuing training from checkpoint, will skip to saved global_step")
        logger.info("  Continuing training from epoch %d", epochs_trained)
        logger.info("  Continuing training from global step %d", global_step)
        logger.info("  Will skip the first %d steps in the first epoch", steps_trained_in_current_epoch)

    tr_loss, logging_loss = 0.0, 0.0
    model.zero_grad()
    train_iterator = trange(
        epochs_trained, int(args.num_train_epochs), desc="Epoch", disable=args.local_rank not in [-1, 0],
    )
    set_seed(args)  # Added here for reproductibility
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=args.local_rank not in [-1, 0])
        for step, batch in enumerate(epoch_iterator):

            # Skip past any already trained steps if resuming training
            if steps_trained_in_current_epoch > 0:
                steps_trained_in_current_epoch -= 1
                continue

            model.train()
            batch = tuple(t.to(args.device) for t in batch)
            inputs = {"input_global_ids": batch[0], "attention_mask_global": batch[2], "input_local_ids": batch[1], "attention_mask_local": batch[3], "text_clean_indices": batch[6], "aspect_indices": batch[7], "labels": batch[8]}
            if args.model_type != "distilbert":
                inputs["token_global_type_ids"] = (
                    batch[4] if args.model_type in ["bert", "xlnet", "albert"] else None
                )
                inputs["token_local_type_ids"] = (
                    batch[5] if args.model_type in ["bert", "xlnet", "albert"] else None
                )
            # inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[3]}
            # if args.model_type != "distilbert":
            #     inputs["token_type_ids"] = (
            #         batch[2] if args.model_type in ["bert", "xlnet", "albert"] else None
            #     )  # XLM, DistilBERT, RoBERTa, and XLM-RoBERTa don't use segment_ids
            outputs = model(**inputs)
            loss = outputs[0]  # model outputs are always tuple in transformers (see doc)

            if args.n_gpu > 1:
                loss = loss.mean()  # mean() to average on multi-gpu parallel training
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            if args.fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()

            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                if args.fp16:
                    torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
                else:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)

                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                if args.local_rank in [-1, 0] and args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    logs = {}
                    if (
                        args.local_rank == -1 and args.evaluate_during_training
                    ):  # Only evaluate when single GPU otherwise metrics may not average well
                        results = evaluate(args, model, tokenizer, tokenizer_cleantext)
                        for key, value in results.items():
                            eval_key = "eval_{}".format(key)
                            logs[eval_key] = value

                    loss_scalar = (tr_loss - logging_loss) / args.logging_steps
                    learning_rate_scalar = scheduler.get_lr()[0]
                    logs["learning_rate"] = learning_rate_scalar
                    logs["loss"] = loss_scalar
                    logging_loss = tr_loss

                    for key, value in logs.items():
                        tb_writer.add_scalar(key, value, global_step)
                    print(json.dumps({**logs, **{"step": global_step}}))

                if args.local_rank in [-1, 0] and args.save_steps > 0 and global_step % args.save_steps == 0:
                    # Save model checkpoint
                    output_dir = os.path.join(args.output_dir, "checkpoint-{}".format(global_step))
                    if not os.path.exists(output_dir):
                        os.makedirs(output_dir)
                    # model_to_save = (
                    #     model.module if hasattr(model, "module") else model
                    # )  # Take care of distributed/parallel training
                    # model_to_save.save_pretrained(output_dir)
                    torch.save(model.state_dict(), args.output_dir)
                    tokenizer.save_pretrained(output_dir)

                    torch.save(args, os.path.join(output_dir, "training_args.bin"))
                    logger.info("Saving model checkpoint to %s", output_dir)

                    torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                    torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
                    logger.info("Saving optimizer and scheduler states to %s", output_dir)

            if args.max_steps > 0 and global_step > args.max_steps:
                epoch_iterator.close()
                break
        if args.max_steps > 0 and global_step > args.max_steps:
            train_iterator.close()
            break

    if args.local_rank in [-1, 0]:
        tb_writer.close()

    return global_step, tr_loss / global_step


def evaluate(args, model, tokenizer, tokenizer_cleantext, prefix=""):
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_task_names = ("ichi",)
    eval_outputs_dirs = (args.output_dir,)

    results = {}
    for eval_task, eval_output_dir in zip(eval_task_names, eval_outputs_dirs):
        eval_dataset, tokenizer_cleantext = load_and_cache_examples(args, eval_task, tokenizer, tokenizer_cleantext, evaluate=True)

        if not os.path.exists(eval_output_dir) and args.local_rank in [-1, 0]:
            os.makedirs(eval_output_dir)

        args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
        # Note that DistributedSampler samples randomly
        eval_sampler = SequentialSampler(eval_dataset)
        eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size)

        # multi-gpu eval
        if args.n_gpu > 1:
            model = torch.nn.DataParallel(model)

        # Eval!
        logger.info("***** Running evaluation {} *****".format(prefix))
        logger.info("  Num examples = %d", len(eval_dataset))
        logger.info("  Batch size = %d", args.eval_batch_size)
        eval_loss = 0.0
        nb_eval_steps = 0
        preds = None
        preds_original = None
        out_label_ids = None
        for batch in tqdm(eval_dataloader, desc="Evaluating"):
            model.eval()
            batch = tuple(t.to(args.device) for t in batch)

            with torch.no_grad():
                inputs = {"input_global_ids": batch[0], "attention_mask_global": batch[2], "input_local_ids": batch[1], "attention_mask_local": batch[3], "text_clean_indices": batch[6], "aspect_indices": batch[7], "labels": batch[8]}
                if args.model_type != "distilbert":
                    inputs["token_global_type_ids"] = (
                        batch[4] if args.model_type in ["bert", "xlnet", "albert"] else None
                    )
                    inputs["token_local_type_ids"] = (
                        batch[5] if args.model_type in ["bert", "xlnet", "albert"] else None
                    )  # XLM, DistilBERT, RoBERTa, and XLM-RoBERTa don't use segment_ids
                outputs = model(**inputs)
                tmp_eval_loss, logits = outputs[:2]

                eval_loss += tmp_eval_loss.mean().item()
            nb_eval_steps += 1
            if preds is None:
                preds = logits.detach().cpu().numpy()
                preds_original = logits.detach().cpu().numpy()
                out_label_ids = inputs["labels"].detach().cpu().numpy()
            else:
                preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
                preds_original = np.append(preds_original, logits.detach().cpu().numpy(), axis=0)
                out_label_ids = np.append(out_label_ids, inputs["labels"].detach().cpu().numpy(), axis=0)

        eval_loss = eval_loss / nb_eval_steps
        preds = np.argmax(preds, axis=1)
        result = compute_metrics(preds, out_label_ids) ######################update kar !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        results.update(result)

        output_eval_file = os.path.join(eval_output_dir, prefix, "eval_results.txt")
        with open(output_eval_file, "w") as writer:
            logger.info("***** Eval results {} *****".format(prefix))
            for key in sorted(result.keys()):
                logger.info("  %s = %s", key, str(result[key]))
                writer.write("%s = %s\n" % (key, str(result[key])))
            # for a in preds:
            #     writer.write(str(a)+'\n')
            writer.write('[')
            for a in preds_original:
                writer.write('[')
                for c in range(len(a)):
                    b = a[c]
                    if c!=6:
                        writer.write(str(b)+',')
                    else:
                        writer.write(str(b))
                writer.write(']')
                writer.write('\n')

    return results


def main():
    parser = argparse.ArgumentParser()

    # Required parameters
    parser.add_argument(
        "--data_dir",
        default=None,
        type=str,
        required=True,
        help="The input data dir. Should contain the .tsv files (or other data files) for the task.",
    )
    parser.add_argument(
        "--model_type",
        default=None,
        type=str,
        required=True,
        help="Model type selected in the list: " + ", ".join(MODEL_CLASSES.keys()),
    )
    parser.add_argument(
        "--model_name_or_path",
        default=None,
        type=str,
        required=True,
        help="Path to pre-trained model or shortcut name selected in the list: " + ", ".join(ALL_MODELS),
    )
    parser.add_argument(
        "--output_dir",
        default=None,
        type=str,
        required=True,
        help="The output directory where the model predictions and checkpoints will be written.",
    )

    # Other parameters
    parser.add_argument(
        "--config_name", default="", type=str, help="Pretrained config name or path if not the same as model_name",
    )
    parser.add_argument(
        "--tokenizer_name",
        default="",
        type=str,
        help="Pretrained tokenizer name or path if not the same as model_name",
    )
    parser.add_argument(
        "--cache_dir",
        default="",
        type=str,
        help="Where do you want to store the pre-trained models downloaded from s3",
    )
    parser.add_argument(
        "--max_seq_length",
        default=128,
        type=int,
        help="The maximum total input sequence length after tokenization. Sequences longer "
        "than this will be truncated, sequences shorter will be padded.",
    )
    parser.add_argument(
        "--embedding_dim_word2vec",
        default=300,
        type=int,
        help="embedding dimension for the word vectors in the medical module",
    )
    parser.add_argument("--do_train", action="store_true", help="Whether to run training.")
    parser.add_argument("--do_eval", action="store_true", help="Whether to run eval on the dev set.")
    parser.add_argument(
        "--evaluate_during_training", action="store_true", help="Rul evaluation during training at each logging step.",
    )
    parser.add_argument(
        "--do_lower_case", action="store_true", help="Set this flag if you are using an uncased model.",
    )

    parser.add_argument(
        "--per_gpu_train_batch_size", default=8, type=int, help="Batch size per GPU/CPU for training.",
    )
    parser.add_argument(
        "--per_gpu_eval_batch_size", default=8, type=int, help="Batch size per GPU/CPU for evaluation.",
    )
    parser.add_argument(
        "--gradient_accumulation_steps",
        type=int,
        default=1,
        help="Number of updates steps to accumulate before performing a backward/update pass.",
    )
    parser.add_argument("--learning_rate", default=5e-5, type=float, help="The initial learning rate for Adam.")
    parser.add_argument("--weight_decay", default=0.0, type=float, help="Weight decay if we apply some.")
    parser.add_argument("--adam_epsilon", default=1e-8, type=float, help="Epsilon for Adam optimizer.")
    parser.add_argument("--max_grad_norm", default=1.0, type=float, help="Max gradient norm.")
    parser.add_argument(
        "--num_train_epochs", default=3.0, type=float, help="Total number of training epochs to perform.",
    )
    parser.add_argument(
        "--max_steps",
        default=-1,
        type=int,
        help="If > 0: set total number of training steps to perform. Override num_train_epochs.",
    )
    parser.add_argument("--warmup_steps", default=0, type=int, help="Linear warmup over warmup_steps.")

    parser.add_argument("--logging_steps", type=int, default=50, help="Log every X updates steps.")
    parser.add_argument("--save_steps", type=int, default=10000, help="Save checkpoint every X updates steps.")
    parser.add_argument(
        "--eval_all_checkpoints",
        action="store_true",
        help="Evaluate all checkpoints starting with the same prefix as model_name ending and ending with step number",
    )
    parser.add_argument("--no_cuda", action="store_true", help="Avoid using CUDA when available")
    parser.add_argument(
        "--overwrite_output_dir", action="store_true", help="Overwrite the content of the output directory",
    )
    parser.add_argument(
        "--overwrite_cache", action="store_true", help="Overwrite the cached training and evaluation sets",
    )
    parser.add_argument("--seed", type=int, default=42, help="random seed for initialization")

    parser.add_argument(
        "--fp16",
        action="store_true",
        help="Whether to use 16-bit (mixed) precision (through NVIDIA apex) instead of 32-bit",
    )
    parser.add_argument(
        "--fp16_opt_level",
        type=str,
        default="O1",
        help="For fp16: Apex AMP optimization level selected in ['O0', 'O1', 'O2', and 'O3']."
        "See details at https://nvidia.github.io/apex/amp.html",
    )
    parser.add_argument("--local_rank", type=int, default=-1, help="For distributed training: local_rank")
    parser.add_argument("--server_ip", type=str, default="", help="For distant debugging.")
    parser.add_argument("--server_port", type=str, default="", help="For distant debugging.")
    args = parser.parse_args()

    if (
        os.path.exists(args.output_dir)
        and os.listdir(args.output_dir)
        and args.do_train
        and not args.overwrite_output_dir
    ):
        raise ValueError(
            "Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(
                args.output_dir
            )
        )

    # Setup distant debugging if needed
    if args.server_ip and args.server_port:
        # Distant debugging - see https://code.visualstudio.com/docs/python/debugging#_attach-to-a-local-script
        import ptvsd

        print("Waiting for debugger attach")
        ptvsd.enable_attach(address=(args.server_ip, args.server_port), redirect_output=True)
        ptvsd.wait_for_attach()

    # Setup CUDA, GPU & distributed training
    if args.local_rank == -1 or args.no_cuda:
        device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
        args.n_gpu = torch.cuda.device_count()
    else:  # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
        torch.cuda.set_device(args.local_rank)
        device = torch.device("cuda", args.local_rank)
        torch.distributed.init_process_group(backend="nccl")
        args.n_gpu = 1
    args.device = device

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if args.local_rank in [-1, 0] else logging.WARN,
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        args.local_rank,
        device,
        args.n_gpu,
        bool(args.local_rank != -1),
        args.fp16,
    )

    # Set seed
    set_seed(args)

    # Prepare GLUE task
    args.task_name = 'ichi'
    processor = ICHIProcessor()
    args.output_mode = "classification"
    label_list = processor.get_labels()
    num_labels = len(label_list)

    # Load pretrained model and tokenizer
    if args.local_rank not in [-1, 0]:
        torch.distributed.barrier()  # Make sure only the first process in distributed training will download model & vocab

    args.model_type = args.model_type.lower()
    config_class, model_class, tokenizer_class, lcf_model = MODEL_CLASSES[args.model_type]
    config = config_class.from_pretrained(
        args.config_name if args.config_name else args.model_name_or_path,
        num_labels=num_labels,
        finetuning_task=args.task_name,
        cache_dir=args.cache_dir if args.cache_dir else None,
    )
    tokenizer = tokenizer_class.from_pretrained(
        args.tokenizer_name if args.tokenizer_name else args.model_name_or_path,
        do_lower_case=args.do_lower_case,
        cache_dir=args.cache_dir if args.cache_dir else None,
    )
    tokenizer_cleantext = Tokenizer(max_seq_len = 1600, lower=True)
    # model = model_class.from_pretrained(
    #     args.model_name_or_path,
    #     from_tf=bool(".ckpt" in args.model_name_or_path),
    #     config=config,
    #     cache_dir=args.cache_dir if args.cache_dir else None,
    #     force_download=True,
    # )

    args.use_single_bert = False
    args.local_context_focus = 'cdm'
    args.embedding_type = 'glove'

    cached_embeddingmatrix_file = os.path.join(
        args.data_dir,
        "cachedword2vec_{}_{}".format(
            str(args.task_name),
            str("glove"),
        ),
    )
    cached_embeddingmatrix_path = "."
    
    train_dataset, tokenizer_cleantext = load_and_cache_examples(args, args.task_name, tokenizer, tokenizer_cleantext, evaluate=False)
    embedding_matrix = build_embedding_matrix_glove(tokenizer_cleantext.word2idx, args.embedding_dim_word2vec, cached_embeddingmatrix_file, cached_embeddingmatrix_path)
    print(embedding_matrix.shape)
    print(tokenizer_cleantext.word2idx)
    
    model = lcf_model.from_pretrained(
        args.model_name_or_path,
        from_tf=bool(".ckpt" in args.model_name_or_path),
        config=config,
        args=args,
        cache_dir=args.cache_dir if args.cache_dir else None,
    )
    # model = lcf_Roberta.from_pretrained(
    #     "/home/bt1/17CS10037/new_transformers/transformers/roberta-large-pytorch_model.bin",
    #     from_tf=bool(".ckpt" in args.model_name_or_path),
    #     config=config,
    #     args=args,
    #     cache_dir=args.cache_dir if args.cache_dir else None,
    #     force_download=True,
    # )

    if args.local_rank == 0:
        torch.distributed.barrier()  # Make sure only the first process in distributed training will download model & vocab

    model.to(args.device)

    logger.info("Training/evaluation parameters %s", args)
    
    # Training
    if args.do_train:
        # train_dataset, tokenizer_cleantext = load_and_cache_examples(args, args.task_name, tokenizer, tokenizer_cleantext, evaluate=False)
        # matrix = build_embedding_matrix_glove(tokenizer_cleantext.word2idx, args.embedding_dim_word2vec, cached_glove_embeddingmatrix_file, cached_glove_embeddingmatrix_path)
        global_step, tr_loss = train(args, train_dataset, model, tokenizer, tokenizer_cleantext)
        logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

    result = evaluate(args, model, tokenizer, tokenizer_cleantext)
    # Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()
    if args.do_train and (args.local_rank == -1 or torch.distributed.get_rank() == 0):
        # Create output directory if needed
        if not os.path.exists(args.output_dir) and args.local_rank in [-1, 0]:
            os.makedirs(args.output_dir)

        logger.info("Saving model checkpoint to %s", args.output_dir)
        # Save a trained model, configuration and tokenizer using `save_pretrained()`.
        # They can then be reloaded using `from_pretrained()`
        # model_to_save = (
        #     model.module if hasattr(model, "module") else model
        # )  # Take care of distributed/parallel training
        # model_to_save.save_pretrained(args.output_dir)
        model_name = "{}.pt".format(args.model_type)
        torch.save(model.state_dict(), os.path.join(args.output_dir,model_name))
        tokenizer.save_pretrained(args.output_dir)

        # Good practice: save your training arguments together with the trained model
        torch.save(args, os.path.join(args.output_dir, "training_args.bin"))

        # Load a trained model and vocabulary that you have fine-tuned
        # model = model_class.from_pretrained(args.output_dir)
        model = lcf_model(config, args)
        model.load_state_dict(torch.load(os.path.join(args.output_dir,model_name)))
        tokenizer = tokenizer_class.from_pretrained(args.output_dir)
        model.to(args.device)

    # Evaluation
    results = {}
    if args.do_eval and args.local_rank in [-1, 0]:
        tokenizer = tokenizer_class.from_pretrained(args.output_dir, do_lower_case=args.do_lower_case)
        cached_tokenizer_cleantext_file = os.path.join(
            args.data_dir,
            "cachedtokenizer_{}_{}_{}".format(
                list(filter(None, args.model_name_or_path.split("/"))).pop(),
                str(args.embedding_type),
                str(args.task_name),
            ),
        )
        if os.path.exists(cached_tokenizer_cleantext_file):
            print('loading tokenizer from cache:', cached_tokenizer_cleantext_file)
            tokenizer_cleantext = pickle.load(open(cached_tokenizer_cleantext_file, 'rb'))
        checkpoints = [args.output_dir]
        if args.eval_all_checkpoints:
            checkpoints = list(
                os.path.dirname(c) for c in sorted(glob.glob(args.output_dir + "/**/" + WEIGHTS_NAME, recursive=True))
            )
            logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
        logger.info("Evaluate the following checkpoints: %s", checkpoints)
        for checkpoint in checkpoints:
            global_step = checkpoint.split("-")[-1] if len(checkpoints) > 1 else ""
            prefix = checkpoint.split("/")[-1] if checkpoint.find("checkpoint") != -1 else ""

            # model = model_class.from_pretrained(checkpoint)
            # model = lcf_model.from_pretrained(
            #     args.model_name_or_path,
            #     from_tf=bool(".ckpt" in args.model_name_or_path),
            #     config=config,
            #     args=args,
            #     cache_dir=args.cache_dir if args.cache_dir else None,
            # )

            model = lcf_model(config, args)
            model_name = "{}.pt".format(args.model_type)
            model.load_state_dict(torch.load(os.path.join(args.output_dir,model_name)))
            model.to(args.device)
            result = evaluate(args, model, tokenizer, tokenizer_cleantext, prefix=prefix)
            result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
            results.update(result)

    return results


if __name__ == "__main__":
    main()


Overwriting /content/transformers/examples/ichi/run_ichi.py


In [0]:
!python ./examples/ichi/run_ichi.py --model_type bert --model_name_or_path bert-base-uncased --do_lower_case --do_train --do_eval --data_dir ./data/ichi --max_seq_length 256 --per_gpu_eval_batch_size=32 --per_gpu_train_batch_size=8 --learning_rate 2e-5 --num_train_epochs 3 --output_dir ./tmp/ichi_bert_base_new --overwrite_output_dir 

2020-04-11 19:09:22.924603: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
04/11/2020 19:09:24 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
04/11/2020 19:09:25 - INFO - filelock -   Lock 139804199121696 acquired on /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685.lock
04/11/2020 19:09:25 - INFO - filelock -   Lock 139804199121696 released on /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685.lock
04/11/2020 19:09:25 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad02514

In [0]:
!ls

CONTRIBUTING.md		     LICENSE	       src
data			     Makefile	       templates
deploy_multi_version_doc.sh  MANIFEST.in       tests
docker			     notebooks	       tmp
docs			     README.md	       transformers-cli
examples		     requirements.txt  utils
glove.840B.300d.txt	     runs	       valohai.yaml
glove.840B.300d.zip	     setup.cfg
hubconf.py		     setup.py


In [0]:
!ls

bioasq.lip6.fr		      hubconf.py	setup.cfg
biomedicalWordVectors.tar.gz  index.html	setup.py
CONTRIBUTING.md		      index.html.1	src
data			      LICENSE		templates
deploy_multi_version_doc.sh   Makefile		tests
docker			      MANIFEST.in	tmp
docs			      notebooks		transformers-cli
examples		      README.md		utils
glove.840B.300d.txt	      requirements.txt	valohai.yaml
glove.840B.300d.zip	      runs		word2vecTools
